In [6]:
import nengo
import numpy as np
import matplotlib.pyplot as plt
import pompy.models
import pompy.processors
from matplotlib.animation import FuncAnimation
plt.rcParams['animation.html'] = 'html5'

In [9]:
def wind_model_demo(dt=0.01, t_max=100, steps_per_frame=20, seed=1):
    """Set up wind model and animate velocity field with quiver plot.

    Parameters
    ----------
    dt : float
        Simulation timestep.
    t_max : float
        End time to simulate to.
    steps_per_frame: integer
        Number of simulation time steps to perform between animation frames.
    seed : integer
        Seed for random number generator.

    Returns
    -------
    fig : Figure
        Matplotlib figure object.
    ax : AxesSubplot
        Matplotlib axis object.
    anim : FuncAnimation
        Matplotlib animation object.
    """
    rng = np.random.RandomState(seed)
    # define simulation region
    wind_region = models.Rectangle(x_min=0., x_max=100., y_min=-25., y_max=25.)
    # set up wind model
    wind_model = models.WindModel(wind_region, 21, 11, rng=rng)
    # let simulation run for 10s to equilibrate wind model
    for t in np.arange(0, 10, dt):
        wind_model.update(dt)
    # generate figure and attach close event
    fig, ax, title = set_up_figure()
    # create quiver plot of initial velocity field
    vf_plot = ax.quiver(wind_model.x_points, wind_model.y_points,
                        wind_model.velocity_field.T[0],
                        wind_model.velocity_field.T[1], width=0.003)
    # expand axis limits to make vectors at boundary of field visible
    ax.axis(ax.axis() + np.array([-0.25, 0.25, -0.25, 0.25]))
    ax.set_xlabel('x-coordinate / m')
    ax.set_ylabel('y-coordinate / m')
    ax.set_aspect(1)
    fig.tight_layout()

    # define update function
    @update_decorator(dt, title, steps_per_frame, [wind_model])
    def update(i):
        vf_plot.set_UVC(
            wind_model.velocity_field.T[0], wind_model.velocity_field.T[1])
        return [vf_plot]

    # create animation object
    n_frame = int(t_max / (dt * steps_per_frame) + 0.5)
    print(n_frame)
    anim = FuncAnimation(fig, update, n_frame, blit=True)
    return fig, ax, anim

In [10]:
from pompy import demos
fig, ax, anim = demos.wind_model_demo()
plt.close(fig)
anim